In [8]:
from imutils.video import VideoStream
from imutils.video import FPS
from imutils.video import VideoStream
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os

In [9]:
# load the face detector 
protopath = os.path.sep.join(['./facedetector/', "deploy.prototxt"])
modelpath = os.path.sep.join(['./facedetector/',"res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protopath, modelpath)
print("Face Detector Loaded")

# load the embedding model 
embedder = cv2.dnn.readNetFromTorch('openface_nn4.small2.v1.t7')
print("Embedder Loaded")

Face Detector Loaded
Embedder Loaded


In [10]:
recognizer = pickle.loads(open('recognizer.pickle', "rb").read())
print("Recognizer Loaded")
LB = pickle.loads(open('labelsface.pickle', "rb").read())
print("labels Loaded")

Recognizer Loaded
labels Loaded


In [11]:
print("starting video stream")
#vs = VideoStream(src=0).start()
time.sleep(2.0)
cap=cv2.VideoCapture(0)


#For writing the video use the below commented code
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#writer = None
#(h, w) = (None, None)


# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    ret, frame = cap.read()

    # resize the frame to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    
    #if writer is None:
        # store the image dimensions, initialize the video writer,
        # and construct the zeros array
        #(h, w) = frame.shape[:2]
        #writer = cv2.VideoWriter('output.mp4', fourcc, 20,(w, h), True)

    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]

        # filter out weak detections
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]


            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vector = embedder.forward()

            # perform classification to recognize the face
            preds = recognizer.predict_proba(vector)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = LB.classes_[j]
        
            # draw the bounding box of the face along with the
            # associated probability
            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y-10),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    # show the output frame
    #writer.write(frame)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


#writer.release()
cap.release()
cv2.destroyAllWindows()


starting video stream
